# Plot gradient images per layer

In order to make a better overview for the results, load the gradient images for all neurons in a layer and plot them as a single image. There are some practical limits in how large an image can be, so layers have been broken down into groups of 50 neurons.

In [1]:
import tensorflow as tf
import vgg16
import utils
import os
import pandas as pd
from pathlib2 import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from __future__ import print_function
import more_itertools
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
images_root_dir = 'input/tiny-imagenet-200/val/images/'
images_output_dir = 'output_images_deconvrelu/'

In [3]:
df_max_n_activations = pd.read_csv('max_n_activations.csv', index_col=[0, 1, 2])

In [ ]:
img_path = next(Path(images_output_dir).glob("*.jpg"))
img = plt.imread(str(img_path))
single_img_height = img.shape[0]
single_img_width = img.shape[1]
dpi = 72 # Matplotlib expects figure sizes in inches, dpi is the conversion between pixel and real size
         # 72 is the default value anyway

In [ ]:
layer_group_size = 50
for layer_name in df_max_n_activations.index.levels[0]:
    print()
    print(layer_name)
    all_filters_for_layer = df_max_n_activations.loc[layer_name].index.get_level_values(0).unique()
    group = 0
    for filters_for_layer in more_itertools.chunked(all_filters_for_layer, layer_group_size): # Break large layers up per 50 filters
        num_of_filters = len(filters_for_layer)
        fig = plt.figure(figsize=(single_img_width / dpi + 1, single_img_height / dpi * num_of_filters / 3.6))
        for filter_num in filters_for_layer:
            filename = images_output_dir + "max_act_images_{}_filter_{}.jpg".format(
                layer_name.replace('/', '_'), filter_num)
            if not os.path.exists(filename):
                continue # the image file has not yet been genearted

            image = plt.imread(filename)
            ax = plt.subplot(num_of_filters / 2, 2, filter_num + 1 - group * layer_group_size)
            plt.axis('off')
            plt.imshow(image)
            plt.title("Layer {} filter {}".format(layer_name, filter_num))
        plt.savefig(images_output_dir + "all_max_act_images_{}_group_{}".format(layer_name.replace('/', '_'), group),
                   bbox_inches='tight', pad_inches = 0)
        group += 1
        print(group, end=" ")
        del(fig)